In [1]:
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import re
import traceback
from warnings import filterwarnings
from IPython.display import Image, display
import matplotlib.pyplot as plt
filterwarnings('ignore')

In [2]:
driver = None

In [3]:
def open_main_screen():
    global driver
    path = "C:\\Program Files (x86)\\chromedriver.exe"
    service = Service(ChromeDriverManager().install())
    options = Options()
    #options.add_argument("--headless")
    chrome_options = Options()
    chrome_options.add_argument("--disable-notifications")  # Bildirimleri devre dışı bırak
    chrome_options.add_argument("--disable-popup-blocking") # Pop-up'ları engelle
    chrome_options.add_argument("--start-maximized")       # Tam ekran başlat
    
    # Tarayıcı profili
    prefs = {"profile.default_content_setting_values.notifications": 2}  # Bildirim izinlerini kapat
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.trendyol.com/"
    driver.get(url)
    
    try:
        cookies_button_rejected = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
        cookies_button_rejected.click()
    except Exception as e:
        print('cookies not found')
        print(e)


In [4]:
keys = [
    'searched_key_word',
    'brand',
    'title',
    'find_dim',
    'find_dim_from_content',
    'guess_dimensions',
    'guess_dim_from_comment',
    'image_path',
]

df = pd.DataFrame(columns=keys)

In [5]:
def search_items(entry):
    global driver
    search_box = driver.find_element(By.XPATH,'//*[@id="sfx-discovery-search-suggestions"]/div/div/input')
    search_box.clear()
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  

In [6]:
def select_most_reated():
    global driver
    recommended = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/div/div')
    recommended.click()
    most_rated = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/ul/li[last()]/span')
    most_rated.click()
    

In [7]:
def find_dim():
    global driver
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/section/div/div/div/ul') 
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        if texts_with_numbers == []:
            texts_with_numbers = [text.strip() for text in relevant_details if re.search(r"(\d+)\s*(cm|mm)", text)]
        return texts_with_numbers
    except Exception as e:
        return f"An error occurred: {str(e)}" # every product must contain these field so if it's not return me a error.

In [8]:
def find_dim_from_content():
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim','ebat']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH, '//*[@id="urun-aciklamasi"]/div')
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        if texts_with_numbers == []:
            texts_with_numbers = [text.strip() for text in relevant_details if re.search(r"(\d+)\s*(cm|mm)", text)]
        return texts_with_numbers
    except Exception as e:
        return None #every product don't have to contain these field so if it's not, note return me any thing

In [9]:
def guess_dimensions():
    global driver
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-reviews-btn')
    except:
        return 'there is no comment'
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(1)
    #driver.execute_script("window.scrollBy(0, -400);") bu biraz daha kesin olmayan yol
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
    except:
        try:
            button = driver.find_element(By.XPATH, "//button[contains(@class, 'tags-wrapper__button') and not(contains(@class, 'rotate-180'))]")
            button.click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
        except:
            return None

    time.sleep(1)
    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer
            
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r"(\d+)\s*(cm|mm)", answer):
            return answer

    return None

In [10]:
def guess_dim_from_comment(entry = 'Boyut' ):
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-questions-btn')
    except:
        return 'there is no comment'
    element.click()
    time.sleep(2)
    search_box = driver.find_element(By.XPATH,'//*[@id="questions-and-answers-app"]/div/div/div/div/div/div[3]/div[2]/div/div[1]/input')
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)

    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer

    for answer in answers:
       answer = answer.text.split('\n')[1]
       answer = answer.lower()
       if re.search(r"(\d+)\s*(cm|mm)", answer):
           return answer
    return None
    

In [11]:
def click_all_features():
    global driver
    ozellikler = driver.find_element(By.CLASS_NAME,'button-all-features')
    ozellikler.click()

In [12]:
#windows = driver.window_handles
#driver.switch_to.window(windows[1])

In [13]:
def close_understood():
    global is_it_first_time
    if is_it_first_time:
        global driver
        understood = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/div[2]/div/div[2]/div[2]/div/div[1]/aside/div[1]/div/div[2]/div/div[2]/div/div/button')
        understood.click()
        is_it_first_time = False

In [14]:
def close_understood2():
    global is_it_first_time2
    if is_it_first_time2:
        global driver
        understood = driver.find_element(By.XPATH,'//*[@id="rating-and-review-app"]/div/div/div/div[3]/div/div/div[3]/div[1]/div/div[2]/div[2]/div[1]/button')
        understood.click()
        is_it_first_time2 = False

In [15]:
def scrap_brand():
    global driver
    brand = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'product-brand-name')]")))
    return brand.text

In [16]:
def scrap_title():
    all_title = driver.find_element(By.CLASS_NAME,'pr-new-br').text
    brand = scrap_brand()
    title  = all_title.strip(brand).strip()
    return brand, title

In [17]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

def scrape_photo(photo_number):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "rvw-cnt-tx"))).click()
    #enter_images = driver.find_element(By.XPATH, "//span[@class='ps-tags__tag--bold' and contains(text(), 'fotoğraflı')]")
    time.sleep(2)
    
    if is_it_first_time2 == True:
        close_understood2()
        
    try:
        enter_images = driver.find_element(By.XPATH,'//*[@id="rating-and-review-app"]/div/div/div/div[3]/div/div/div[3]/div[1]/div/div[1]/button')
        enter_images.click()
    except:
        return "There is no enough photo"

   
    gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
    image_links = []
    #for index in range(0,(int(photo_number/20))): # there are 20 photos in scroll panel when we open that
    for index in range(0,photo_number):
        if index % 20==0 and index !=0:
            gallery = driver.find_element(By.CLASS_NAME, "image-gallery-content")
            gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
            driver.execute_script("arguments[0].scrollTop += 400;", gallery)
            time.sleep(1)
            driver.execute_script("arguments[0].scrollTop += 400;", gallery)
            time.sleep(1)
            driver.execute_script("arguments[0].scrollTop += 100;", gallery)
            time.sleep(3)
            
    gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
    for item in gallery_items:
        img = item.find_element(By.TAG_NAME, "img")
        image_links.append(img.get_attribute('src'))

    return image_links

In [24]:
product = [
    #'Termos',
    #'Su şişesi',
    #'Kupa bardak',
    #'Kalemlik',
    #'Gözlük kabı',
    #'Kulaklık kutusu',
    #'Powerbank',
    #'Mouse',
    #'Klavye',
    #'LEGO figürleri',
    #'Aksiyon figürleri',
    #'Tıraş makinesi',
    #'Parfüm şişesi',
    #'Tarak',
    #'Oje şişesi',
    'tekli baharatlıklar',
    'tekli Mini tencere (1 kişilik)',
    'tek Kalem',
    'tek Defter (küçük boy)',
    'tekli Silgi',
    'Zımba',
    'Yapıştırıcı',
    'tekli Dambıl',
    'Kamp çakmağı',
    'El feneri',
    'Katlanabilir çakı',
    'USB bellek',
    'Mikrofon',
    'Küçük vazo',
    'Biblo',
    'Anahtarlık',
    'Mini drone',
    'Küçük puzzle parçaları',
    'Oyun kartları (iskambil, UNO vb.)',
    'Kutu oyunu taşları',
    'Maket bıçakları',
    'Cüzdan',
    'Kemer tokası',
    'Eldiven',
    'Şapka',
    'Saat kutusu',
    'Lens kutusu',
    'Diş fırçası',
    'Küçük deodorant şişesi',
    'Tırnak makası',
    'Cımbız',
    'Cep aynası',
    'El masaj aleti       ',
    'Küçük kahve fincanı',
    'Şarj adaptörü',
    'Biberon',
    'silah',
    'Gözlük kabı',
    'Küçük el feneri',
    'duvar saati',
    'araç kokusu',
    'kartlık',
    'zeka küpü',
    'Kitap tutucu',
    'Kum saati',
    'cop kovasi',
    'Bluetooth hoparlör',
    'Dijital tartı',
    'el kantari',
    'eldiven',
    'küllük',
    'robot supurge',
    'projeksiyon cihazi',
    'suluk',
    'tekli Mum',
    'deodorant şişesi',
    'ampul',
    ' Ahşap Merdane Oky-bolu-m040'
    'c Easyfix Serisi Cam Temizleme Aparatı 2.863-025.0']

In [19]:
product2 = [
    'Masa lambası',
    'Tablet',
    'Dizüstü bilgisayar',
    'Telefon kabı',
    'Sırt çantası',
    'Gözlük',
    'Anahtarlık',
    'Futbol topu',
    'Bilye',
    'Şemsiye',
    'Maket araba',
    'Yapboz',
    'Kalem',
    'İlk yardım seti',
    'Dikiş seti',
    'Not defteri',
    'Post-it not',
    'Çanta',
    'Büyük boy kutu',
    'Küçük boy kutu',
    'Kıyafet askısı',
    'İp',
    'Sünger',
    'Baskı kağıdı',
    'Yazıcı',
    'Dergi',
    'Küpe',
    'Yüzük',
    'Dikiş iğnesi',
    'Kumaş parçası',
    'Boya fırçası',
    'Ruj',
    'Dudak balmı',
    'Diş ipi',
    'Makası',
    'Köpek tasması',
    'Kedi oyuncakları',
    'Kedi maması',
    'Köpek maması',
    'Paketleme bandı',
    'Zincir',
    'Pasta kalıbı',
    'Fırın tepsisi',
    'Makarna süzgeci',
    'Tava',
    'Saklama kabı',
    'Küçük masa'
]


In [20]:
markalar = [
            'ikea',
            'koctas']

In [25]:
def run():
    global driver
    global is_it_first_time
    is_it_first_time = True
    global is_it_first_time2
    is_it_first_time2 = True
    open_main_screen()
    search_keys = product
    for key in search_keys:
        search_items(key)
        #select_most_reated()
        time.sleep(2)
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'variant-options-wrapper')))
        count = 0
        for i in products:
            i.click()
            time.sleep(2)
            windows = driver.window_handles
            driver.switch_to.window(windows[1])
            if is_it_first_time:
                close_understood()
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            time.sleep(2)
            brand = scrap_title()[0]
            title = scrap_title()[1]
            key_word = key
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            click_all_features()
            time.sleep(1)
            find_dim_ = find_dim()
            driver.execute_script("window.scrollBy(0, 400);") #bazi urunlerde aciklama olmadigi icin ortala yerine kaydiriyorum.
            find_dim_from_content_ = find_dim_from_content()
            guess_dimensions_ = guess_dimensions()
            if guess_dimensions_ == 'there is no comment':
                guess_dimensions_ = None
            else:
                driver.back()
            try: 
                guess_dim_from_comment_ = guess_dim_from_comment()

            except:
                guess_dim_from_comment_ = 'there is no comment'
            
            if guess_dim_from_comment_ == 'there is no comment':
                guess_dim_from_comment_ = None
            else:
                driver.back()
            driver.execute_script("window.scrollTo(0, 0);")
            #buradan sonra fotograf cekme asamalri gelmesi lazim
            if ((isinstance(find_dim_, list) and len(find_dim_) == 0) or (pd.isna(find_dim_).all() if isinstance(find_dim_, (list, pd.Series, np.ndarray)) else pd.isna(find_dim_))) and \
            ((pd.isna(find_dim_from_content_).all()) if isinstance(find_dim_from_content_, (list, pd.Series, np.ndarray)) else pd.isna(find_dim_from_content_)) and \
            ((pd.isna(guess_dimensions_).all()) if isinstance(guess_dimensions_, (list, pd.Series, np.ndarray)) else pd.isna(guess_dimensions_)) and \
            ((pd.isna(guess_dim_from_comment_).all()) if isinstance(guess_dim_from_comment_, (list, pd.Series, np.ndarray)) else pd.isna(guess_dim_from_comment_)):
                
                driver.close()
                driver.switch_to.window(windows[0])
                driver.execute_script("window.scrollTo(0, 0);")

            else:
                try:
                    images = scrape_photo(60)
                    print(f'cekilen fotograf sayisi: {len(images)}')
                    #buradan once fotograf cekme asamalri gelmesi lazim
                    for img in images:
                        df.loc[len(df)] = [key_word, brand ,title, find_dim_, find_dim_from_content_, guess_dimensions_,
                                           guess_dim_from_comment_, img]
                        print(df.tail(1))
                    driver.close()
                    driver.switch_to.window(windows[0])
                    driver.execute_script("window.scrollTo(0, 0);")
                    products = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'variant-options-wrapper')))
                    if count == 15:
                        count = 0
                        break
                    else:
                        count+=1
                except:
                    driver.close()
                    driver.switch_to.window(windows[0])
                    driver.execute_script("window.scrollTo(0, 0);")
                    products = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'variant-options-wrapper')))
                    pass
            

In [26]:
run()

cekilen fotograf sayisi: 57
     searched_key_word        brand  \
5116     Parfüm şişesi  MİMOZA PARK   

                                                  title find_dim  \
5116  arfüm Şişesi Çanta içi Anahtarlıklı Mini Doldu...       []   

     find_dim_from_content guess_dimensions guess_dim_from_comment  \
5116                    []     6x10 efendim           6x10 efendim   

                                             image_path  
5116  https://review-rating.mncdn.com/mnresize/140/2...  
     searched_key_word        brand  \
5117     Parfüm şişesi  MİMOZA PARK   

                                                  title find_dim  \
5117  arfüm Şişesi Çanta içi Anahtarlıklı Mini Doldu...       []   

     find_dim_from_content guess_dimensions guess_dim_from_comment  \
5117                    []     6x10 efendim           6x10 efendim   

                                             image_path  
5117  https://review-rating.mncdn.com/mnresize/140/2...  
     searched_key_word    

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00650283+25139]
	(No symbol) [0x005DB234]
	(No symbol) [0x004B0300]
	(No symbol) [0x004EB1B8]
	(No symbol) [0x0051CDF6]
	(No symbol) [0x005187E5]
	(No symbol) [0x00517D66]
	(No symbol) [0x00483245]
	(No symbol) [0x0048379E]
	(No symbol) [0x00483C1D]
	GetHandleVerifier [0x0095B143+3215603]
	GetHandleVerifier [0x009722BA+3310186]
	GetHandleVerifier [0x0096C4D2+3286146]
	GetHandleVerifier [0x006E9C80+654384]
	(No symbol) [0x005E45BD]
	(No symbol) [0x00482F10]
	(No symbol) [0x00482AD6]
	GetHandleVerifier [0x009E6E5C+3788300]
	BaseThreadInitThunk [0x76985D49+25]
	RtlInitializeExceptionChain [0x77ACCDEB+107]
	RtlGetAppContainerNamedObjectPath [0x77ACCD71+561]


In [28]:
df.to_csv('2.3data.csv',index = False)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df

In [ ]:
print('hello worldsddd')